### Imports

In [1]:
from simcse import SimCSE
import pandas as pd
import numpy as np
from pathlib import Path

### Load model and dataset

In [2]:
model = SimCSE('./data/dental_model/')

Some weights of BertModel were not initialized from the model checkpoint at ./data/dental_model/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
dataset = pd.read_csv('data/input/cluster_simcse_input.csv', index_col=0)

### Process

In [21]:
dataset

,row,column,sentence
0,0,0,Schmerzen im Kiefergelenk
1,0,1,Schmerzfreiheit
2,0,2,"essen vor allem harte, zähe Speisen, wenn das ..."
3,0,3,Essen
4,0,4,wenn der Kiefer entspannt ist
...,...,...,...
61707,2127,24,NaN
61708,2127,25,NaN
61709,2127,26,NaN
61710,2127,27,NaN


In [23]:
sentence_values = [str(i) for i in list(dataset['sentence'].values)]

In [18]:
sentence_embedding = model.encode(sentence_values)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 965/965 [01:19<00:00, 12.08it/s]


In [19]:
len(sentence_embedding[0])

768

In [29]:
sts_clutser_embeddings = []

# Loop both embeddings and enter them in the new array
for i in range(dataset.shape[0]):
    sts_clutser_embeddings.append([
        dataset['row'][i],
        dataset['column'][i],
        dataset['sentence'][i],
        np.array(sentence_embedding[i])
    ])

In [30]:
sts_clutser_embeddings[:5]

[[0,
  0,
  'Schmerzen im Kiefergelenk',
  array([ 3.40620205e-02,  1.39922276e-02,  1.42524922e-02, -3.04563660e-02,
          2.88767908e-02,  3.35586742e-02,  1.73320752e-02,  3.55277322e-02,
         -6.47233278e-02, -4.35374910e-03, -4.83951531e-02, -1.47298835e-02,
          2.60293223e-02, -5.16897291e-02,  5.10057360e-02,  1.07752755e-02,
          4.15190719e-02, -4.87493277e-02, -2.06252858e-02, -2.88332012e-02,
         -3.22713517e-02,  5.78450505e-03, -6.47987574e-02,  3.04630678e-03,
         -2.45785434e-02,  3.31313908e-02, -1.41197694e-02, -3.14958915e-02,
          5.07663609e-03,  1.50573086e-02,  9.67024709e-04, -4.35582735e-02,
         -2.84598451e-02, -3.35769169e-02,  5.77964634e-02, -9.37123690e-03,
          2.06972249e-02,  1.24295624e-02,  4.53915354e-03, -8.58539715e-03,
          2.95980461e-03, -4.20897454e-03,  6.38315082e-02, -4.73546423e-02,
          1.51535328e-02, -6.32033776e-03,  5.57578504e-02, -1.03816669e-03,
         -1.81395262e-02,  5.384533

In [31]:
sts_cluster_input = pd.DataFrame(sts_clutser_embeddings)
sts_cluster_input.columns = ['row','column','sentence','embedding']

In [32]:
sts_cluster_input

,row,column,sentence,embedding
0,0,0,Schmerzen im Kiefergelenk,"[0.03406202, 0.013992228, 0.014252492, -0.0304..."
1,0,1,Schmerzfreiheit,"[-0.07252122, 0.026934693, -0.04297495, 0.0319..."
2,0,2,"essen vor allem harte, zähe Speisen, wenn das ...","[-0.020250382, 0.08037725, 0.008891177, -0.054..."
3,0,3,Essen,"[-0.06400962, 0.014543581, 0.015365832, 0.0073..."
4,0,4,wenn der Kiefer entspannt ist,"[-0.00921524, 0.050368443, 0.02311897, 0.03054..."
...,...,...,...,...
61707,2127,24,NaN,"[-0.033935856, -0.020590236, 0.02946174, 0.032..."
61708,2127,25,NaN,"[-0.033935856, -0.020590236, 0.02946174, 0.032..."
61709,2127,26,NaN,"[-0.033935856, -0.020590236, 0.02946174, 0.032..."
61710,2127,27,NaN,"[-0.033935856, -0.020590236, 0.02946174, 0.032..."


In [33]:
# Check folder exists
Path('./data/output').mkdir(parents=True, exist_ok=True)

# Export pandas dataframe to .csv
sts_cluster_input.to_csv('data/output/dental_cluster_input.csv')